### Fintech - MSc BAM - Project

Packages installation

In [3]:
#Package installation
import requests
import networkx as nx
import random
import pandas as pd
import sqlite3
import math
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from fastapi.responses import JSONResponse
import uvicorn
from networkx import all_simple_paths
import nest_asyncio
from pyngrok import ngrok

In [4]:
DEBUG_MODE = 1  # Set to 0 for production mode - allows to simulate executions without a user input to allow for unit testing

Define supported currencies

In [6]:
# Define the supported currencies
FIAT_CURRENCIES = [
    'USD', 'EUR', 'GBP', 'CAD', 'AUD', 'JPY', 'MAD', 'ZAR', 'INR', 'BRL',
    'TRY', 'HUF', 'MXN', 'THB', 'NGN', 'COP', 'PEN' , 'SGD', 'HKD',  
    'PLN',  
    'CZK',  
    'DKK',  
    'SEK',  
    'NOK',  
    'CLP',  
    'ARS',  
    'UAH' 
]

CRYPTO_CURRENCIES = [
    'BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'ADA', 'AVAX', 'XMR', 'MATIC', 'TRX', 'LTC', 'NEAR', 'DOGE','SHIB','LINK', 'XLM', 'ATOM', 'AAVE',
    'FTM','ARB','OP','UNI'
]

SUPPORTED_CURRENCIES = FIAT_CURRENCIES + CRYPTO_CURRENCIES

#Map tikker to coingecko id's 
COINGECKO_ID_MAP = {
    'BTC': 'bitcoin', 'ETH': 'ethereum', 'USDT': 'tether',
    'BNB': 'binancecoin', 'SOL': 'solana', 'ADA': 'cardano',
    'AVAX': 'avalanche-2', 'XMR': 'monero', 'MATIC': 'matic-network',
    'TRX': 'tron', 'LTC': 'litecoin', 'NEAR': 'near','DOGE': 'dogecoin',
    'SHIB': 'shiba-inu', 'LINK': 'chainlink', 'XLM': 'stellar', 'ATOM': 'cosmos',
    'AAVE': 'aave', 'FTM': 'fantom', 'ARB': 'arbitrum', 'OP': 'optimism','UNI': 'uniswap'
}

#Finds the top 5 hubs in a networkx graph based on degree centrality 
def identify_hubs(G, top_n=5):
    return [node for node, degree in sorted(G.degree, key=lambda x: x[1], reverse=True)[:top_n]]


Create apis for all currency relationships

In [8]:
# Function to add noise to the rate for non-hub edges
def add_noise(rate, is_hub_edge):
    if is_hub_edge:
        return rate  # No noise for hub routes
    noise = random.uniform(0.01, 0.05)  # 1–5% positive
    return rate * (1 + noise)


In [9]:
#API call to fetch crypto rates in USD
def fetch_crypto_usd_rates():
    ids = ','.join(COINGECKO_ID_MAP.values()) # Get all Coingecko IDs
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={ids}&vs_currencies=usd"
    r = requests.get(url)
    data = r.json() #Gets the rates in JSON format
    return {
        symbol: data.get(id_, {}).get('usd')
        for symbol, id_ in COINGECKO_ID_MAP.items()
        if data.get(id_, {}).get('usd')
    } #extracts the rates for each symbol in USD

#API call to fetch fiat rates in USD
def fetch_usd_fiat_rates():
    url = "https://open.er-api.com/v6/latest/USD"
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception(f"Failed to fetch USD fiat rates. Status: {r.status_code}") #Error handling for failed API call
    data = r.json()
    if 'rates' not in data:
        raise Exception(f"Invalid response: {data}")
    return {
        fiat: data['rates'][fiat]
        for fiat in FIAT_CURRENCIES
        if fiat in data['rates']
    } #Returns a dictionary of fiat currencies and their rates in USD

#API call to fetch crypto to crypto rates
def fetch_crypto_crypto_rates():
    symbols = list(COINGECKO_ID_MAP.keys())
    ids = ','.join(COINGECKO_ID_MAP.values())
    vs_currencies = ','.join(symbols)

    url = f"https://api.coingecko.com/api/v3/simple/price?ids={ids}&vs_currencies={vs_currencies}"
    r = requests.get(url)
    data = r.json()

    rates = {}
    for base_symbol, base_id in COINGECKO_ID_MAP.items():
        for quote_symbol in symbols:
            if quote_symbol != base_symbol:
                rate = data.get(base_id, {}).get(quote_symbol.lower())
                if rate:
                    rates[(base_symbol, quote_symbol)] = rate
    return rates #Returns a dictionary of crypto to crypto rates if the crypto is supported and is not the same as the base currency

#API call to fetch fiat to fiat rates
def build_fiat_to_fiat_rates(usd_fiat_rates):
    rates = {}
    for base in FIAT_CURRENCIES:
        for quote in FIAT_CURRENCIES:
            if base != quote:
                try:
                    base_to_usd = 1 / usd_fiat_rates[base]
                    usd_to_quote = usd_fiat_rates[quote]
                    rate = base_to_usd * usd_to_quote
                    rates[(base, quote)] = rate
                except KeyError:
                    continue
    return rates


Spin up network graph for shortest path

In [11]:
# Function to build a full currency graph with spread and noise and ensuring unidirectional edges

"""
This function builds a directed graph representing currency exchange rates with spread and noise. Noise was added to non-hub edges to simulate arbitrage opportunities. 
The graph is built using the provided exchange rates for cryptocurrencies and fiat currencies, and it identifies hubs based on degree centrality. 
The spread is applied differently for hub edges and non-hub edges, with noise added to non-hub edges to create a more realistic trading environment.
The spread is needed to simulate the cost of trading. Unidirectional edges only are enforced to represent one way trades as it is illogical to 
hop back and forth between two currencies in a single trade when servicing payments.
"""

def build_full_currency_graph_with_spread_unidirectional(
    crypto_usd_rates,
    usd_fiat_rates,
    crypto_crypto_rates,
    fiat_fiat_rates,
    base_spread_pct=0.000025,
    hub_spread_pct=0.9,
    epsilon=1e-4
):
    G = nx.DiGraph() #Base graph initialisation

    #A function to determine the spread based on whether the edge is a hub edge or not
    def edge_spread(u, v, hubs):
        return hub_spread_pct if u in hubs or v in hubs else base_spread_pct

    #A function to add noise to the rate based on whether the edge is a hub edge or not
    def add_noise(rate, is_hub_edge):
        if is_hub_edge:
            return rate
        return rate * random.uniform(1.5, 2.5)  # Controlled arbitrage emergence being enforced to simulate arbitrage opportunities

    # Build graph normally first, using base_spread
    #Adds crypto to USD
    for crypto, base_rate in crypto_usd_rates.items():
        if base_rate:
            G.add_edge(crypto, 'USD', rate=base_rate, weight=-math.log(base_rate + epsilon))

    # Adds USD to fiat
    for fiat, base_rate in usd_fiat_rates.items():
        if base_rate:
            G.add_edge('USD', fiat, rate=base_rate, weight=-math.log(base_rate + epsilon))

    #Adds crypto to crypto rates
    for (u, v), base_rate in crypto_crypto_rates.items():
        if base_rate:
            G.add_edge(u, v, rate=base_rate, weight=-math.log(base_rate + epsilon))

    #Adds fiat to fiat rates
    for (u, v), base_rate in fiat_fiat_rates.items():
        if base_rate:
            G.add_edge(u, v, rate=base_rate, weight=-math.log(base_rate + epsilon))

    # Identify hubs based on degree
    hubs = identify_hubs(G, top_n=5)

    # Rebuild graph with spread + noise
    G_final = nx.DiGraph()

    for u, v, data in G.edges(data=True):
        base_rate = data['rate']
        is_hub_edge = u in hubs or v in hubs
        spread = hub_spread_pct if is_hub_edge else base_spread_pct
        noisy_rate = add_noise(base_rate, is_hub_edge)
        bid = noisy_rate * (1 - spread)
        G_final.add_edge(u, v, rate=bid, weight=-math.log(bid + epsilon))

    print(f"Unidirectional graph built with {len(G_final.nodes())} nodes and {len(G_final.edges())} edges")
    return G_final


In [12]:
# Helper to run Bellman-Ford with a single arbitrage fix allowed

"""
This function implements the Bellman-Ford algorithm to find the shortest path in a directed graph, while also handling negative cycles.
It attempts to correct negative cycles by adjusting edge weights, allowing for a maximum number of arbitrary corrections.
The maximum number of corrections is set to 1 by default, meaning it will only attempt to fix one negative cycle before raising an exception.
If it is not fixed to one, it will continuously adjust the edges for the negative cycle, making unrealitic profits from arbitrage opportunities.
"""

def safe_bellman_ford_path(G, source, target, max_arb_corrections=1):
    def find_negative_cycle(G):
        distance = {node: float('inf') for node in G.nodes()} #Finds the distance from the source to each node
        predecessor = {}
        start = list(G.nodes())[0] #Picks the first node as the start node
        distance[start] = 0 #Sets the distance from the start node to itself as 0

        for _ in range(len(G.nodes()) - 1):
            for u, v, data in G.edges(data=True):
                weight = data['weight']
                if distance[u] + weight < distance[v]: #Distance from U with weight is less than the distance from V 
                    distance[v] = distance[u] + weight #Set distance from V to the distance from U plus the weight to stop negative cycles
                    predecessor[v] = u

        for u, v, data in G.edges(data=True): #for each edge in the graph, a check is made to see if the distance from U plus the weight is less than the distance from V
            if distance[u] + data['weight'] < distance[v]:
                cycle = [v]
                while u not in cycle:
                    cycle.append(u)
                    u = predecessor[u]
                cycle.append(u)
                cycle.reverse()
                return cycle
        return None

    try_count = 0 #Sets the try count to 0 for first iteration
    while try_count <= max_arb_corrections: #checks to see if the try count is less than or equal to the max arbitrage corrections
        try:
            path = nx.bellman_ford_path(G, source=source, target=target, weight='weight') #applies the Bellman-Ford algorithm to find the shortest path from source to target
            return path
        except:
            cycle = find_negative_cycle(G) #applies the find_negative_cycle function to find a negative cycle in the graph
            if cycle and try_count < max_arb_corrections:
                u, v = cycle[0], cycle[1]
                G[u][v]['weight'] += 1e-3  # Penalize to break cycle
                try_count += 1
            else:
                break
    raise Exception("Negative cycle could not be corrected.")

Locate negative cycle to stop infinite loop in Bellman Ford

Get direct rate for quote

In [15]:
#Function to get the direct rate between two currencies, if it exists, or through a hub. 
def get_direct_rate(buyer, seller, graph):
    if graph.has_edge(buyer, seller):
        return graph[buyer][seller]['rate']
    for hub in HUBS:
        try:
            rate_buyer_to_hub = graph[buyer][hub]['rate']
            rate_hub_to_seller = graph[hub][seller]['rate']
            return rate_buyer_to_hub * rate_hub_to_seller
        except KeyError:
            continue
    return None

Run the full optimisation

In [17]:
"""
A function to enrich transactions with optimization, which includes:
1. Direct rate calculation between buyer and seller.
2. Attempting to find an optimized path using the Bellman-Ford algorithm.
3. Handling negative cycles by capturing arbitrage profits and correcting the graph.
4. Calculating final amounts after applying commissions and arbitrage profits.
"""

def enrich_transactions_with_optimization(transactions_df, graph, commission_split=0.5):
    enriched = []
    
    for _, row in transactions_df.iterrows():
        buyer, seller, amount = row['buyer_currency'], row['seller_currency'], row['amount']
        arbitrage_profit = 0

        direct_rate = get_direct_rate(buyer, seller, graph) #Finds direct rate between buyer and seller
        direct_final = amount * direct_rate if direct_rate else None

        # --- Try Optimization ---
        try:
            path = nx.bellman_ford_path(graph, source=buyer, target=seller, weight='weight') # Finds the shortest path from buyer to seller using the Bellman-Ford algorithm
            optimized_final = amount
            for i in range(len(path) - 1):
                optimized_final *= graph[path[i]][path[i+1]]['rate'] #Finds the optimized final amount by multiplying the rates along the path
            optimized_rate = optimized_final / amount

        except Exception as e:
            if "Negative cycle" in str(e):
                print(f"Negative cycle detected during {buyer} → {seller}. Capturing arbitrage profit.")

                # 1. Find the negative cycle
                cycle = find_negative_cycle(graph)
                if cycle:
                    print(f"Arbitrage cycle: {' → '.join(cycle)}")

                    # 2. Calculate arbitrage profit factor
                    profit_factor = 1.0
                    for i in range(len(cycle) - 1):
                        u, v = cycle[i], cycle[i+1]
                        profit_factor *= graph[u][v]['rate']

                    if profit_factor > 1: #if profit is more than profit factor of 1, then arbitrage profit is possible due to negative cycle
                        arbitrage_profit = (profit_factor - 1) * amount
                        print(f"Arbitrage profit: {arbitrage_profit:.6f}")

                    # 3. Correct the graph (kill the cycle)
                    u, v = cycle[0], cycle[1]
                    graph[u][v]['weight'] += 1e-4  # Slightly worsen an edge - arbitary amount to break the cycle

                    # 4. Retry optimization in new iteration
                    try:
                        path = safe_bellman_ford_path(G, source=from_currency, target=to_currency)
                        optimized_final = amount
                        for i in range(len(path) - 1):
                            optimized_final *= graph[path[i]][path[i+1]]['rate']
                        optimized_rate = optimized_final / amount #Finds optimized rate by dividing the optimized final amount by the original amount
                    except: #If the optimization fails again, then set path, optimized_final and optimized_rate to None
                        path = None
                        optimized_final = None
                        optimized_rate = None
                else: # If no cycle is found set path, optimized_final and optimized_rate to None
                    path = None
                    optimized_final = None
                    optimized_rate = None
            else:
                print(f"Unexpected error optimizing {buyer} → {seller}: {e}")
                path = None
                optimized_final = None
                optimized_rate = None

        # --- Calculate final commissions ---
        if optimized_final and direct_final:
            savings = (optimized_final - direct_final)
            commission = (savings * commission_split if savings > 0 else 0) + arbitrage_profit
            final_amount = optimized_final - (commission if savings > 0 else 0)
        elif direct_final:  # fallback to direct final
            savings = 0
            commission = arbitrage_profit  # only arbitrage if any
            final_amount = direct_final - arbitrage_profit
        else:
            savings = 0
            commission = 0
            final_amount = None

        enriched.append({**row,
            'optimized_path': ' → '.join(path) if path else None,
            'direct_rate': round(direct_rate, 6) if direct_rate else None,
            'optimized_rate': round(optimized_rate, 6) if optimized_rate else None,
            'commission_earned': round(commission, 6) if commission else 0,
            'final_seller_amount': round(final_amount, 6) if final_amount else None
        })
    
    return pd.DataFrame(enriched)



Convert commission to USD for quanitifiable revenue

In [19]:
#Find all commissions in USD for each transaction in the enriched DataFrame to allow for easy comparison of commissions in different currencies.
def convert_commissions_to_usd(enriched_df, graph):
    commissions_usd = []
    for _, row in enriched_df.iterrows():
        seller_currency = row['seller_currency']
        commission = row['commission_earned']
        if pd.isna(commission) or commission == 0:
            commissions_usd.append(0)
            continue
        try:
            if seller_currency == 'USD':
                commission_usd = commission
            else:
                rate_to_usd = graph[seller_currency]['USD']['rate']
                commission_usd = commission * rate_to_usd
        except:
            commission_usd = None
        commissions_usd.append(commission_usd)
    enriched_df['commission_earned_usd'] = commissions_usd
    return enriched_df


Create connection to remote react.js server to send data to front end

In [21]:
#React frontend setup: This is used to allow the frontend to connect to the backend API
!ngrok config add-authtoken 2xYAmGxTQ020GH7Q4sZrhzRPVSq_7SEXzAsQiuhyhJEgtd34V #hardcoded ngrok token for the frontend to connect to the backend API - must be replaced with your own token
#ngrok token is used to allow the frontend to connect to the backend API

#If debug mode is not enabled, run the FastAPI server with ngrok to allow for easy access to the API from the frontend.
if DEBUG_MODE == 0:
    def run_engine(amount, from_currency, to_currency):
        crypto_usd = fetch_crypto_usd_rates()
        fiat_usd = fetch_usd_fiat_rates()
        crypto_crypto = fetch_crypto_crypto_rates()
        fiat_fiat = build_fiat_to_fiat_rates(fiat_usd)
        G = build_full_currency_graph_with_spread_unidirectional(
            crypto_usd_rates=crypto_usd,
            usd_fiat_rates=fiat_usd,
            crypto_crypto_rates=crypto_crypto,
            fiat_fiat_rates=fiat_fiat,
            base_spread_pct=0.0025, # 0.25% base spread for all edges
            hub_spread_pct=0.01  # or tweak this value as needed to toggle spread on hub edges
        )

        # Direct rate quote
        direct_rate = get_direct_rate(from_currency, to_currency, G)
        if not direct_rate:
            raise Exception("Conversion path not found")
    
        direct_amount = round(amount * direct_rate, 2) #Rounded direct amount to two decimal places
    
        # Initialize variables for optimization
        arbitrage_profit = 0
        optimized_amount = direct_amount
        optimized_rate = round(direct_rate, 6)
        path = [from_currency, to_currency]
    
        # Attempt to optimize path (multi-hop)
        try:
            path = safe_bellman_ford_path(G, source=from_currency, target=to_currency)
            optimized_amount = amount
            for i in range(len(path) - 1): #for each node in the path, multiply the amount by the rate of the edge to get the optimized amount
                optimized_amount *= G[path[i]][path[i+1]]['rate']
            optimized_amount = round(optimized_amount, 2)
            optimized_rate = round(optimized_amount / amount, 6)
        except Exception as e:
            if "Negative cycle" in str(e):
                cycle = find_negative_cycle(G) #Finds a negative cycle in the graph
                if cycle:
                    print(f"Arbitrage cycle found: {' → '.join(cycle)}")
                    profit_factor = 1.0 # The profit factor is set to 1 representing the initial value where no profit is made
                    for i in range(len(cycle) - 1):
                        u, v = cycle[i], cycle[i + 1]
                        profit_factor *= G[u][v]['rate'] #Derives profit factor by multiplying the rates of the edges in the cycle
                    if profit_factor > 1: #if profit factor is greater than 1, then arbitrage profit is made
                        arbitrage_profit = round((profit_factor - 1) * amount, 2) #arbitrage profit is calculated by subtracting 1 from the profit factor and multiplying it by the amount
            else:
                print(f"Optimization failed: {e}")
    
        savings = round(optimized_amount - direct_amount, 2) #Savings are calculated by subtracting the direct amount from the optimized amount
        commission = round((savings * 0.5 if savings > 0 else 0) + arbitrage_profit, 2) #Commission is calculated by taking half of the savings if savings are greater than 0, otherwise it is 0, and adding the arbitrage profit
        final_amount = round(optimized_amount - commission, 2) #Round the final amount to two decimal places by subtracting the commission from the optimized amount
    
        return {
            "direct_amount": direct_amount,
            "converted_amount": final_amount,
            "optimized_rate": optimized_rate,
            "exchange_path": path,
            "savings": savings,
            "commission": commission,
            "arbitrage_profit": arbitrage_profit
        }
    
    # FastAPI initialization
    app = FastAPI()
    app.add_middleware(CORSMiddleware, allow_origins=['*'], allow_methods=['*'], allow_headers=['*'])
    
    # Define the API request and response models
    class ConvertRequest(BaseModel):
        amount: float
        from_currency: str
        to_currency: str
    
    #Define the API connection to the conversion engine
    @app.post("/api/convert")
    def convert(req: ConvertRequest):
        try:
            return run_engine(req.amount, req.from_currency, req.to_currency)
        except Exception as e:
            return JSONResponse(status_code=500, content={"error": str(e)})
    
    nest_asyncio.apply() # Allows nested event loops for Jupyter compatibility
    public_url = ngrok.connect(8000) # Connects to ngrok to expose the FastAPI server
    print("Public FastAPI URL for frontend:", public_url)
    uvicorn.run(app, port=8000)

Authtoken saved to configuration file: C:\Users\benja\AppData\Local/ngrok/ngrok.yml


In [22]:
#If debug mode is enabled, the code will run in a simulated environment to allow for unit testing and debugging without user input.
if DEBUG_MODE:
    print("DEBUG MODE: Searching for profitable multi-hop routes...\n")

    test_amount = 1  # Simulated base amount
    profitable_cases = []

    # Build graph as usual
    crypto_usd = fetch_crypto_usd_rates()
    fiat_usd = fetch_usd_fiat_rates()
    crypto_crypto = fetch_crypto_crypto_rates()
    fiat_fiat = build_fiat_to_fiat_rates(fiat_usd)

    G = build_full_currency_graph_with_spread_unidirectional(
        crypto_usd_rates=crypto_usd,
        usd_fiat_rates=fiat_usd,
        crypto_crypto_rates=crypto_crypto,
        fiat_fiat_rates=fiat_fiat,
        base_spread_pct=0.000025,
        hub_spread_pct=0.9 # or tweak this value as needed
    )

    print(f"Graph built with {len(G.nodes())} nodes and {len(G.edges())} edges\n")

    for source in G.nodes():
        for target in G.nodes():
            if source == target or not G.has_edge(source, target):
                continue

            try:
                # Canonical direct rate
                direct_rate = G[source][target]['rate']
                direct_amount = test_amount * direct_rate

                # Best (possibly multi-hop) path
                path = nx.bellman_ford_path(G, source, target, weight='weight')
                multi_hop_amount = test_amount
                for i in range(len(path) - 1):
                    multi_hop_amount *= G[path[i]][path[i+1]]['rate']

                if direct_amount and len(path) > 2 and multi_hop_amount > direct_amount * 1.005:
                    profitable_cases.append((source, target, path, direct_amount, multi_hop_amount))

            except Exception:
                continue

    # Show top N results
    if profitable_cases:
        print(f"🔍 Found {len(profitable_cases)} profitable multi-hop cases:\n")
        for source, target, path, direct_amt, multi_amt in sorted(profitable_cases, key=lambda x: x[-1] - x[-2], reverse=True)[:5]:
            print(f"{source} → {target}")
            print(f"Direct: {direct_amt:.2f}, Multi-hop: {multi_amt:.2f}")
            print(f"Path: {' → '.join(path)}\n")
            print("Top hubs used for spread penalty:", hubs)
    else:
        print("No profitable multi-hop routes found over direct ones.")


🛠️ DEBUG MODE: Searching for profitable multi-hop routes...

✅ Unidirectional graph built with 49 nodes and 872 edges
✅ Graph built with 49 nodes and 872 edges

❌ No profitable multi-hop routes found over direct ones.
